# Exploring Hacker News Posts

Hacker News is a site where user-submitted stories (known as "posts") receive votes and comments, similar to reddit. It is extremely popular within technology and startup circles and popular posts can get hundres of thousands of visitors.

The data set can be found at https://www.kaggle.com/datasets/hacker-news/hacker-news-postsand 

Seen below are descriptions of the columns

| Column | Explanation
| --- | ---|
|id | the unique identifier from Hacker News for the post |
|title | the title of the post |
|url | the URL that the posts links to, if the post has a URL |
|num_points | the number of points the post acquired, calculated as the total number of upvotes minus the total number of downvotes|
|num_comments | the number of comments on the post |
|author | the username of the person who submitted the post |
|created_at | the date and time of the post's submission |

In [1]:
from csv import reader
open_file = open("hacker_news.csv")
read_file = reader(open_file)
hn = list(read_file)

headers = hn[0]
hn = hn[1:]

print(headers)
print("\n")
print(hn[:5])

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']


[['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01'], ['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']]


Let's divide up the posts into 3 lists, ask hn, show hn, and other

In [2]:
ask_posts = []
show_posts = []
other_posts = []

for row in hn:
    title = row[1]
    title_lower = title.lower()
    if title_lower.startswith('ask hn'):
        ask_posts.append(row)
    elif title_lower.startswith('show hn'):
        show_posts.append(row)
    else:
        other_posts.append(row)
        
print("Number of Ask Posts:", len(ask_posts))
print("Number of Show Posts:", len(show_posts))
print("Number of Other Posts:", len(other_posts))

Number of Ask Posts: 1744
Number of Show Posts: 1162
Number of Other Posts: 17194


Let's examine the first 5 rows of ask_posts list

In [3]:
print(ask_posts[:5])

[['12296411', 'Ask HN: How to improve my personal website?', '', '2', '6', 'ahmedbaracat', '8/16/2016 9:55'], ['10610020', 'Ask HN: Am I the only one outraged by Twitter shutting down share counts?', '', '28', '29', 'tkfx', '11/22/2015 13:43'], ['11610310', 'Ask HN: Aby recent changes to CSS that broke mobile?', '', '1', '1', 'polskibus', '5/2/2016 10:14'], ['12210105', 'Ask HN: Looking for Employee #3 How do I do it?', '', '1', '3', 'sph130', '8/2/2016 14:20'], ['10394168', 'Ask HN: Someone offered to buy my browser extension from me. What now?', '', '28', '17', 'roykolak', '10/15/2015 16:38']]


Let's examine the first 5 rows of show_posts list

In [4]:
print(show_posts[:5])

[['10627194', 'Show HN: Wio Link  ESP8266 Based Web of Things Hardware Development Platform', 'https://iot.seeed.cc', '26', '22', 'kfihihc', '11/25/2015 14:03'], ['10646440', 'Show HN: Something pointless I made', 'http://dn.ht/picklecat/', '747', '102', 'dhotson', '11/29/2015 22:46'], ['11590768', 'Show HN: Shanhu.io, a programming playground powered by e8vm', 'https://shanhu.io', '1', '1', 'h8liu', '4/28/2016 18:05'], ['12178806', 'Show HN: Webscope  Easy way for web developers to communicate with Clients', 'http://webscopeapp.com', '3', '3', 'fastbrick', '7/28/2016 7:11'], ['10872799', 'Show HN: GeoScreenshot  Easily test Geo-IP based web pages', 'https://www.geoscreenshot.com/', '1', '9', 'kpsychwave', '1/9/2016 20:45']]


Let's now determine if ask posts or show posts receive more comments on average.

In [5]:
total_ask_comments = 0
for row in ask_posts:
    num_comments = int(row[4])
    total_ask_comments += num_comments 

avg_ask_comments = total_ask_comments / len(ask_posts)
print("Average comments for Ask Posts:", avg_ask_comments)


total_show_comments = 0
for row in show_posts:
    num_comments = int(row[4])
    total_show_comments += num_comments 

avg_show_comments = total_show_comments / len(show_posts)
print("Average comments for Show Posts:", avg_show_comments)

Average comments for Ask Posts: 14.038417431192661
Average comments for Show Posts: 10.31669535283993


As we can see from this result ask posts recieve more comments on average and is therfore the dataset we will examine and analyze further.

Let's now determine if ask posts created at a certain time are more likely to attract more comments. Using 2 steps.

The first step is to calculate the number of ask posts created in each hour of the day, along with the number of comments received.

The second step is to calculate the average number of comments ask posts recieve by hour created.

In [6]:
import datetime as dt
result_list = []
for row in ask_posts:
    created_at = row[6]
    num_comments = int(row[4])
    result_list.append([created_at, num_comments])

#Contains the number of ask posts created during each hour of the day 
counts_by_hour = {}
#Contains the corresponding number of comments ask posts created at each hour recieved
comments_by_hour = {}
for row in result_list:
    date = row[0]
    comment = row[1]
    parse_data = dt.datetime.strptime(date, "%m/%d/%Y %H:%M")
    hour_passed = parse_data.strftime("%H")
    if hour_passed not in counts_by_hour:
        counts_by_hour[hour_passed] = 1
        comments_by_hour[hour_passed] = comment
    else:
        counts_by_hour[hour_passed] += 1
        comments_by_hour[hour_passed] += comment
        
    
    
    

    
    

Now that we have our two dictionaries lets calculate the average number of comments for posts created during each hour of the day and store the results in a list

In [9]:
#Creating a function that gives the average for each hour
def average(dict_1, dict_2):
    average_by_hour = []
    
    for key in dict_1:
        average = dict_2[key]/dict_1[key]
        average_by_hour.append([key, average])
    return average_by_hour


avg_by_hour = average(counts_by_hour, comments_by_hour)
print(avg_by_hour)

[['09', 5.5777777777777775], ['13', 14.741176470588234], ['10', 13.440677966101696], ['14', 13.233644859813085], ['16', 16.796296296296298], ['23', 7.985294117647059], ['12', 9.41095890410959], ['17', 11.46], ['15', 38.5948275862069], ['21', 16.009174311926607], ['20', 21.525], ['02', 23.810344827586206], ['18', 13.20183486238532], ['03', 7.796296296296297], ['05', 10.08695652173913], ['19', 10.8], ['01', 11.383333333333333], ['22', 6.746478873239437], ['08', 10.25], ['04', 7.170212765957447], ['00', 8.127272727272727], ['06', 9.022727272727273], ['07', 7.852941176470588], ['11', 11.051724137931034]]


Although we now have the results we need, this format makes it difficult to identify the hours with the highest values. Let's finish by sorting the list of lists and printing the five highest values in a format that's easier to read.

In [18]:
swap_avg_by_hour = []
for row in avg_by_hour:
    hour = row[0]
    avg = row[1]
    swap_avg_by_hour.append([avg, hour])
    
sorted_swap = sorted(swap_avg_by_hour, reverse=True)
print("Top 5 Hours for Ask Posts Comments")

for row in sorted_swap[:5]:
    avg = round(row[0], 2)
    hour = row[1]
    format_date = dt.datetime.strptime(hour, "%H")
    format_date = format_date.strftime("%H:%M:")
    print(format_date, avg, "average comments per post")

Top 5 Hours for Ask Posts Comments
15:00: 38.59 average comments per post
02:00: 23.81 average comments per post
20:00: 21.52 average comments per post
16:00: 16.8 average comments per post
21:00: 16.01 average comments per post


# Conclusion

Therefore the results show that firstly Ask Posts receive more comments on average for Hacker News

Exploring the Ask Posts dataset it is identified in the results that 15:00 24hr time or 3:00 12 hour time is the time where most comments are received on average. The also show the late afternoon and the evening are the best times to recieve more comments. This may be due to the fact that most people work day jobs and will not have the time to comment untill after work or school. 